In [1]:
!pip install autogluon

  Obtaining dependency information for autogluon from https://files.pythonhosted.org/packages/2e/42/f26592ecf3dc4e4edcbc5f7f7e3deba25b0681fad2ad04a82d12af17e3a4/autogluon-0.8.2-py3-none-any.whl.metadata
  Obtaining dependency information for autogluon.core[all]==0.8.2 from https://files.pythonhosted.org/packages/e0/56/545adb1d388e78591cd7e36de0c8b889c1944de362bdaeec0f31d01890df/autogluon.core-0.8.2-py3-none-any.whl.metadata
  Obtaining dependency information for autogluon.features==0.8.2 from https://files.pythonhosted.org/packages/bb/ea/7892719f78a30aee1bf42c4a0540fbae98bfbdf56b85fab79ffc437eb687/autogluon.features-0.8.2-py3-none-any.whl.metadata
  Obtaining dependency information for autogluon.tabular[all]==0.8.2 from https://files.pythonhosted.org/packages/f3/dc/0bd8cadb9a5e2f3e5b12caaa6745357d912ffc7b8b75fb4e426a38331028/autogluon.tabular-0.8.2-py3-none-any.whl.metadata
  Obtaining dependency information for autogluon.multimodal==0.8.2 from https://files.pythonhosted.org/packages/f

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
import pandas as pd
import numpy as np
import os
import random
from tqdm.auto import tqdm
# import optuna

# DATA_PATH = '/content/drive/MyDrive/데이터 분석/데이콘 캐글 컴페티션/예슬 개인 참가/제주 특산물 가격 예측/data/'
DATA_PATH = '/kaggle/input/Jeju-dataset/'
# DATA_PATH = "C:\python-code\DACON_JEJU\data/"
SEED = 42

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(SEED) # Seed 고정

In [4]:
import numpy as snp
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings(action='ignore')

In [5]:
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn import tree
# from catboost import CatBoostRegressor
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

from sklearn.model_selection import KFold, train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error, make_scorer

```
item : 품목 코드
TG : 감귤
BC : 브로콜리
RD : 무
CR : 당근
CB : 양배추
corporation : 유통 법인 코드
법인 A부터 F 존재
location : 지역 코드
J : 제주도 제주시
S : 제주도 서귀포시
supply(kg) : 유통된 물량, kg 단위
price(원/kg) : 유통된 품목들의 kg 마다의 가격, 원 단위
```

# DATA Load

In [6]:
DATA_PATH = "/kaggle/input/jeju-dataset/"

train_df = pd.read_csv(f"{DATA_PATH}train.csv")
test_df = pd.read_csv(f"{DATA_PATH}test.csv")
international = pd.read_csv(f"{DATA_PATH}international_trade.csv")
submission = pd.read_csv(f"{DATA_PATH}sample_submission.csv")

In [7]:
train_df.rename(columns={'supply(kg)':'supply', 'price(원/kg)':'price'}, inplace=True)

In [8]:
train_df['item_id'] = train_df.ID.str[0:6]
test_df['item_id'] = test_df.ID.str[0:6]


# Preprocessing

In [9]:
# 공휴일
holi_weekday = ['2019-01-01', '2019-02-04', '2019-02-05', '2019-02-06', '2019-03-01', '2019-05-05', '2019-05-12', '2019-06-06', '2019-08-15', '2019-09-12', '2019-09-13', '2019-09-14', '2019-10-03', '2019-10-09', '2019-12-25',
                '2020-01-01' ,'2020-01-24' ,'2020-01-25', '2020-01-26', '2020-03-01', '2020-04-30', '2020-05-05', '2020-06-06', '2020-08-15', '2020-08-17', '2020-09-30', '2020-10-01', '2020-10-02', '2020-10-03', '2020-10-09', '2020-12-25',
                '2021-01-01' ,'2021-02-11' ,'2021-02-12', '2021-02-13', '2021-03-01', '2021-05-05', '2021-05-19', '2021-06-06', '2021-08-15', '2021-09-20', '2021-09-21', '2021-09-22', '2021-10-03', '2021-10-09', '2021-12-25',
                '2022-01-01' ,'2022-01-31' ,'2022-02-01', '2022-02-02', '2022-03-01', '2022-05-05', '2022-05-08', '2022-06-06', '2022-08-15', '2022-09-09', '2022-09-10', '2022-09-11', '2022-09-12', '2022-10-03', '2022-10-09', '2020-10-10', '2022-12-25',
                '2023-01-01' ,'2023-01-21' ,'2023-01-22', '2023-01-23', '2023-01-24', '2023-03-01']

In [10]:
train_df['timestamp'] = pd.to_datetime(train_df['timestamp'])
train_df['year'] = train_df['timestamp'].dt.year
train_df['month_ori'] = train_df['timestamp'].dt.month
train_df['week'] = train_df['timestamp'].dt.isocalendar().week
train_df['day'] = train_df['timestamp'].dt.day
train_df['day_of_week'] = train_df['timestamp'].dt.dayofweek
# train_df['sin_week'] = np.sin(2 * np.pi * train_df['week'] / 4)
# train_df['cos_week'] = np.cos(2 * np.pi * train_df['week'] / 4)
# train_df['sin_dayofweek'] = np.sin(2 * np.pi * (train_df['day_of_week'] + 1) / 7)
# train_df['cos_dayofweek'] = np.cos(2 * np.pi * (train_df['day_of_week'] + 1) / 7)
train_df['holiday'] = np.where((train_df.day_of_week >= 5) | (train_df.timestamp.dt.strftime('%Y-%m-%d').isin(holi_weekday)), 1, 0)


test_df['timestamp'] = pd.to_datetime(test_df['timestamp'])
test_df['year'] = test_df['timestamp'].dt.year
test_df['month_ori'] = test_df['timestamp'].dt.month
test_df['week'] = test_df['timestamp'].dt.isocalendar().week
test_df['day'] = test_df['timestamp'].dt.day
test_df['day_of_week'] = test_df['timestamp'].dt.dayofweek
# test_df['sin_week'] = np.sin(2 * np.pi * test_df['week'] / 4)
# test_df['cos_week'] = np.cos(2 * np.pi * test_df['week'] / 4)
# test_df['sin_dayofweek'] = np.sin(2 * np.pi * (test_df['day_of_week'] + 1) / 7)
# test_df['cos_dayofweek'] = np.cos(2 * np.pi * (test_df['day_of_week'] + 1) / 7)
test_df['holiday'] = np.where((test_df.day_of_week >= 5) | (test_df.timestamp.dt.strftime('%Y-%m-%d').isin(holi_weekday)), 1, 0)

In [11]:

import math

# 연중일 주기
def cos_year(date):
    day_of_year = date.timetuple().tm_yday  # 해당 날짜의 연중 몇 번째 날인지를 가져옵니다.
    period = 365  # 1년은 365일
    return math.cos(2 * math.pi * day_of_year / period)

def sin_year(date):
    day_of_year = date.timetuple().tm_yday  # 해당 날짜의 연중 몇 번째 날인지를 가져옵니다.
    period = 365  # 1년은 365일
    return math.sin(2 * math.pi * day_of_year / period)

train_df['cos_year'] = train_df['timestamp'].apply(cos_year)
train_df['sin_year'] = train_df['timestamp'].apply(sin_year)

test_df['cos_year'] = test_df['timestamp'].apply(cos_year)
test_df['sin_year'] = test_df['timestamp'].apply(sin_year)

# # 요일 주기 (주간)
# def cos_weekday(date):
#     day_of_week = date.weekday()  # Monday is 0, Sunday is 6
#     period = 7  # 1 week has 7 days
#     return math.cos(2 * math.pi * day_of_week / period)

# def sin_weekday(date):
#     day_of_week = date.weekday()  # Monday is 0, Sunday is 6
#     period = 7  # 1 week has 7 days
#     return math.sin(2 * math.pi * day_of_week / period)

# train_df['cos_dow'] = train_df['timestamp'].apply(cos_weekday)
# train_df['sin_dow'] = train_df['timestamp'].apply(sin_weekday)

# test_df['cos_dow'] = test_df['timestamp'].apply(cos_weekday)
# test_df['sin_dow'] = test_df['timestamp'].apply(sin_weekday)

# 연중주 주기
def cos_week_of_year(date):
    week_of_year = date.isocalendar()[1]  # Get the ISO week number
    period = 52  # 1 year has 52 weeks
    return math.cos(2 * math.pi * week_of_year / period)

def sin_week_of_year(date):
    week_of_year = date.isocalendar()[1]  # Get the ISO week number
    period = 52  # 1 year has 52 weeks
    return math.sin(2 * math.pi * week_of_year / period)

train_df['cos_week_of_year'] = train_df['timestamp'].apply(cos_week_of_year)
train_df['sin_week_of_year'] = train_df['timestamp'].apply(sin_week_of_year)

test_df['cos_week_of_year'] = test_df['timestamp'].apply(cos_week_of_year)
test_df['sin_week_of_year'] = test_df['timestamp'].apply(sin_week_of_year)

## 월별 품목의 가격 추가(mean, max)

In [12]:
train_df['month_of_year'] = train_df['timestamp'].dt.to_period('M')
test_df['month_of_year'] = test_df['timestamp'].dt.to_period('M')

In [16]:
train_df

,ID,timestamp,item,corporation,location,supply,price,item_id,year,month_ori,week,day,day_of_week,holiday,cos_year,sin_year,cos_week_of_year,sin_week_of_year,month_of_year
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,TG_A_J,2019,1,1,1,1,1,0.999852,0.017213,0.992709,0.120537,2019-01
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,TG_A_J,2019,1,1,2,2,0,0.999407,0.034422,0.992709,0.120537,2019-01
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,TG_A_J,2019,1,1,3,3,0,0.998667,0.051620,0.992709,0.120537,2019-01
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,TG_A_J,2019,1,1,4,4,0,0.997630,0.068802,0.992709,0.120537,2019-01
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,TG_A_J,2019,1,1,5,5,1,0.996298,0.085965,0.992709,0.120537,2019-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,468.0,RD_F_J,2023,2,9,27,0,0,0.541628,0.840618,0.464723,0.885456,2023-02
59393,RD_F_J_20230228,2023-02-28,RD,F,J,421980.0,531.0,RD_F_J,2023,2,9,28,1,0,0.527078,0.849817,0.464723,0.885456,2023-02
59394,RD_F_J_20230301,2023-03-01,RD,F,J,382980.0,574.0,RD_F_J,2023,3,9,1,2,1,0.512371,0.858764,0.464723,0.885456,2023-03
59395,RD_F_J_20230302,2023-03-02,RD,F,J,477220.0,523.0,RD_F_J,2023,3,9,2,3,0,0.497513,0.867456,0.464723,0.885456,2023-03


In [ ]:
item_month_price = train_df.groupby(['item_id', 'month_of_year'])
agg = {'price' : ['mean', 'max']}
price_df = item_month_price.agg(agg).reset_index()

In [ ]:
price_df.columns = ['item_id', 'month_of_year', 'month_price_mean', 'month_price_max']

train_df = pd.merge(train_df, price_df, on=['item_id', 'month_of_year'], how='left')
test_df = pd.merge(test_df, price_df, on=['item_id', 'month_of_year'], how='left')


## 월별 품목의 물량 추가(mean, max)

- 전달 물량 추가 / 3개월 물량 추가

,ID,timestamp,item,corporation,location,supply,price,item_id,year,month_ori,week,day,day_of_week,holiday,cos_year,sin_year,cos_week_of_year,sin_week_of_year,month_of_year
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,TG_A_J,2019,1,1,1,1,1,0.999852,0.017213,0.992709,0.120537,2019-01
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,TG_A_J,2019,1,1,2,2,0,0.999407,0.034422,0.992709,0.120537,2019-01
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,TG_A_J,2019,1,1,3,3,0,0.998667,0.051620,0.992709,0.120537,2019-01
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,TG_A_J,2019,1,1,4,4,0,0.997630,0.068802,0.992709,0.120537,2019-01
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,TG_A_J,2019,1,1,5,5,1,0.996298,0.085965,0.992709,0.120537,2019-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,468.0,RD_F_J,2023,2,9,27,0,0,0.541628,0.840618,0.464723,0.885456,2023-02
59393,RD_F_J_20230228,2023-02-28,RD,F,J,421980.0,531.0,RD_F_J,2023,2,9,28,1,0,0.527078,0.849817,0.464723,0.885456,2023-02
59394,RD_F_J_20230301,2023-03-01,RD,F,J,382980.0,574.0,RD_F_J,2023,3,9,1,2,1,0.512371,0.858764,0.464723,0.885456,2023-03
59395,RD_F_J_20230302,2023-03-02,RD,F,J,477220.0,523.0,RD_F_J,2023,3,9,2,3,0,0.497513,0.867456,0.464723,0.885456,2023-03


In [39]:
# # item_month_supply = train_df.groupby(['item', 'month_of_year'])
# item_month_supply = train_df.groupby(['item_id', 'month_of_year'])

# agg = {'supply' : ['mean', 'max']}
# supply_df = item_month_supply.agg(agg).reset_index()

In [40]:
# supply_df.columns = ['item_id', 'month_of_year', 'month_supply_mean', 'month_supply_max']

# train_df = pd.merge(train_df, supply_df, on=['item_id', 'month_of_year'], how='left')
# test_df = pd.merge(test_df, supply_df, on=['item_id', 'month_of_year'], how='left')


In [41]:
# 공급과 가격의 상관관계
sup_price_corr = train_df['supply'].corr(train_df['price'])
sup_price_corr

-0.0022967366004861545

## international 데이터 merge

In [26]:
international = pd.read_csv(f"{DATA_PATH}international_trade.csv")

In [27]:
international = international.rename(columns={
    '기간': 'period',
    '품목명': 'item',
    '수출 중량': 'exWeight',
    '수출 금액': 'exValue',
    '수입 중량': 'imWeight',
    '수입 금액': 'imValue',
    '무역수지': 'tradeBalance'
})

international.head()

,period,item,exWeight,exValue,imWeight,imValue,tradeBalance
0,2019-01,토마토(신선한 것이나 냉장한 것으로 한정한다),356571,990,0,0,990
1,2019-01,양파,821330,222,4003206,1118,-896
2,2019-01,쪽파,60,1,93405,128,-127
3,2019-01,꽃양배추와 브로콜리(broccoli),160,1,638913,563,-562
4,2019-01,방울다다기 양배추,0,0,7580,38,-38


In [28]:
international['year'] = international['period'].apply(lambda x : int(x[0:4]))
international['month_ori'] = international['period'].apply(lambda x : int(x[5:7]))
international.drop(columns = 'period', inplace=True)
international

,item,exWeight,exValue,imWeight,imValue,tradeBalance,year,month_ori
0,토마토(신선한 것이나 냉장한 것으로 한정한다),356571,990,0,0,990,2019,1
1,양파,821330,222,4003206,1118,-896,2019,1
2,쪽파,60,1,93405,128,-127,2019,1
3,꽃양배추와 브로콜리(broccoli),160,1,638913,563,-562,2019,1
4,방울다다기 양배추,0,0,7580,38,-38,2019,1
...,...,...,...,...,...,...,...,...
1269,포포(papaw)[파파야(papaya)],0,0,23830,71,-71,2023,2
1270,사과,135165,351,0,0,351,2023,2
1271,배,2206012,5411,1,0,5411,2023,2
1272,신 체리[프루너스 체라서스(Prunus cerasus)],5,0,0,0,0,2023,2


In [32]:
international.loc[international['item'].str.contains('감귤|배|복숭아|사과|살구|수박|자두|참외'), 'category'] = 'ex_fruit'
international.loc[international['item'].str.contains('그레이프|망고|오렌지|파인애플|체리'), 'category'] = 'im_fruit'

international.loc[international['item'].str.contains('토마토|순무|(?!다다기,꽃)양배추|양파|오이'), 'category'] = 'ex_vege'
international.loc[international['item'].str.contains('상추|구아바|브로콜리|당근|레몬|무화과|방울다다기|샬롯|양파|치커리|쪽파|파파야'), 'category'] = 'im_vege'

# 완두, 콩, 대추야자  제외 (과일이나 채소 아닌 곡류) 
international_type = international.groupby(['category','year','month_ori'])[['exWeight','exValue','imWeight','imValue','tradeBalance']].sum().reset_index()

In [22]:
international_type

,category,year,month_ori,exWeight,exValue,imWeight,imValue,tradeBalance
0,ex_fruit,2019,1,3635002,10297,1500,19,10277
1,ex_fruit,2019,2,1083795,2748,26380,257,2491
2,ex_fruit,2019,3,1170348,3188,22159,164,3024
3,ex_fruit,2019,4,544789,1406,13457,36,1370
4,ex_fruit,2019,5,397765,1122,0,0,1122
...,...,...,...,...,...,...,...,...
195,im_vege,2022,10,251568,288,11785495,7492,-7205
196,im_vege,2022,11,44266,53,8347745,7046,-6993
197,im_vege,2022,12,27937,48,13154816,8214,-8167
198,im_vege,2023,1,18866,45,9114212,6431,-6387


In [33]:
train_df.loc[train_df['item'].str.contains('TG'), 'category'] = 'ex_fruit'
train_df.loc[train_df['item'].str.contains('CB|RD'), 'category'] = 'ex_vege'
train_df.loc[train_df['item'].str.contains('BC|CR'), 'category'] = 'ex_fruit'

train_df.loc[train_df['item'].str.contains('TG'), 'type'] = 1
train_df.loc[train_df['item'].str.contains('CB|RD|BC|CR'), 'type'] = 0

test_df.loc[test_df['item'].str.contains('TG'), 'category'] = 'ex_fruit'
test_df.loc[test_df['item'].str.contains('CB|RD'), 'category'] = 'ex_vege'
test_df.loc[test_df['item'].str.contains('BC|CR'), 'category'] = 'ex_fruit'

test_df.loc[test_df['item'].str.contains('TG'), 'type'] = 1
test_df.loc[test_df['item'].str.contains('CB|RD|BC|CR'), 'type'] = 0

In [34]:
mask = international_type['category'] == 'ex_fruit'
international_type[mask]

,category,year,month_ori,exWeight,exValue,imWeight,imValue,tradeBalance
0,ex_fruit,2019,1,3635002,10297,1500,19,10277
1,ex_fruit,2019,2,1083795,2748,26380,257,2491
2,ex_fruit,2019,3,1170348,3188,22159,164,3024
3,ex_fruit,2019,4,544789,1406,13457,36,1370
4,ex_fruit,2019,5,397765,1122,0,0,1122
5,ex_fruit,2019,6,94102,241,0,0,241
6,ex_fruit,2019,7,169058,667,1,0,667
7,ex_fruit,2019,8,4894807,14579,16069,63,14516
8,ex_fruit,2019,9,3478592,9858,32005,84,9775
9,ex_fruit,2019,10,6021636,16209,35795,91,16118


In [35]:
train_df.merge(international_type, on=['category','year','month_ori'], how='left',suffixes = ('','_category') )
test_df.merge(international_type, on=['category','year','month_ori'], how='left',suffixes = ('','_category') )

,ID,timestamp,item,corporation,location,item_id,year,month_ori,week,day,...,imWeight,imValue,tradeBalance,exPrice,imPrice,exWeight_category,exValue_category,imWeight_category,imValue_category,tradeBalance_category
0,TG_A_J_20230304,2023-03-04,TG,A,J,TG_A_J,2023,3,9,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TG_A_J_20230305,2023-03-05,TG,A,J,TG_A_J,2023,3,9,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TG_A_J_20230306,2023-03-06,TG,A,J,TG_A_J,2023,3,10,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TG_A_J_20230307,2023-03-07,TG,A,J,TG_A_J,2023,3,10,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TG_A_J_20230308,2023-03-08,TG,A,J,TG_A_J,2023,3,10,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1087,RD_F_J_20230327,2023-03-27,RD,F,J,RD_F_J,2023,3,13,27,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1088,RD_F_J_20230328,2023-03-28,RD,F,J,RD_F_J,2023,3,13,28,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1089,RD_F_J_20230329,2023-03-29,RD,F,J,RD_F_J,2023,3,13,29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1090,RD_F_J_20230330,2023-03-30,RD,F,J,RD_F_J,2023,3,13,30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1]:
train_df

NameError: name 'train_df' is not defined

In [29]:
selected_items = international[international['item'].str.contains('감귤|브로콜리|무|당근|양배추')].copy()
selected_items

mask = (selected_items['item'] == '무화과') | (selected_items['item'] == '방울다다기 양배추')
selected_items = selected_items[~mask]
selected_items

,item,exWeight,exValue,imWeight,imValue,tradeBalance,year,month_ori
3,꽃양배추와 브로콜리(broccoli),160,1,638913,563,-562,2019,1
5,양배추,184650,94,395802,90,4,2019,1
8,당근,23150,22,7466150,2955,-2934,2019,1
17,감귤,58368,172,0,0,172,2019,1
28,꽃양배추와 브로콜리(broccoli),780,1,396870,399,-398,2019,2
...,...,...,...,...,...,...,...,...
1248,꽃양배추와 브로콜리(broccoli),24,0,332640,352,-352,2023,2
1250,양배추,13188,13,377456,104,-91,2023,2
1253,당근,22510,20,9260020,3758,-3737,2023,2
1254,순무,4000,4,2,0,4,2023,2


In [30]:
selected_items['item'] = selected_items['item'].replace({
    '감귤': 'TG',
    '브로콜리': 'BC',
    '무': 'RD',
    '당근': 'CR',
    '양배추': 'CB'
}, regex = True)

selected_items['item'] = selected_items['item'].str.replace('.*감귤.*', 'TG')
selected_items['item'] = selected_items['item'].str.replace('.*브로콜리.*', 'BC')
selected_items['item'] = selected_items['item'].str.replace('.*무.*', 'RD')
selected_items['item'] = selected_items['item'].str.replace('.*당근.*', 'CR')
selected_items['item'] = selected_items['item'].str.replace('.*양배추.*', 'CB')

In [31]:
train_df = train_df.merge(selected_items, on=['year', 'month_ori', 'item'], how='left')
test_df = test_df.merge(selected_items, on=['year', 'month_ori', 'item'], how='left')

train_df['exValue'] = train_df['exValue']*1000
train_df['imValue'] = train_df['exValue']*1000


# price항목은 있고 없고 동일함
train_df['exPrice'] = train_df['exValue'] / train_df['exWeight']
train_df['imPrice'] = train_df['imValue'] / train_df['imWeight']

test_df['exPrice'] = test_df['exValue'] / test_df['exWeight']
test_df['imPrice'] = test_df['imValue'] / test_df['imWeight']

train_df['exWeight'] = np.log(train_df['exWeight'])
train_df['imWeight'] = np.log(train_df['imWeight'])
train_df['exValue'] = np.log(train_df['exValue'])
train_df['imValue'] = np.log(train_df['imValue'])

test_df['exWeight'] = np.log(test_df['exWeight'])
test_df['imWeight'] = np.log(test_df['imWeight'])
test_df['exValue'] = np.log(test_df['exValue'])
test_df['imValue'] = np.log(test_df['imValue'])

In [54]:
train_df = train_df.fillna(0)
test_df = test_df.fillna(0)

train_df.isnull().sum().sum() , test_df.isnull().sum().sum()

(0, 0)

## 기타 피쳐추가

# Encoding

## 원핫

In [55]:
# # item OneHotEncoding
# train_df = pd.get_dummies(train_df, columns=['item'])
# test_df = pd.get_dummies(test_df, columns=['item'])

## 라벨

In [56]:
obj_cols = ['item', 'corporation', 'location']

for col in obj_cols:
    enc = LabelEncoder()
    train_df[col]=enc.fit_transform(train_df[col])
    test_df[col]=enc.transform(test_df[col])

train_df.head()

,ID,timestamp,item,corporation,location,supply,price,item_id,year,month_ori,...,month_price_mean,month_price_max,exWeight,exValue,imWeight,imValue,tradeBalance,category,exPrice,imPrice
0,TG_A_J_20190101,2019-01-01,4,0,0,0.0,0.0,TG_A_J,2019,1,...,1513.516129,2932.0,10.974523,12.05525,-inf,18.963005,172.0,0.0,2.94682,inf
1,TG_A_J_20190102,2019-01-02,4,0,0,0.0,0.0,TG_A_J,2019,1,...,1513.516129,2932.0,10.974523,12.05525,-inf,18.963005,172.0,0.0,2.94682,inf
2,TG_A_J_20190103,2019-01-03,4,0,0,60601.0,1728.0,TG_A_J,2019,1,...,1513.516129,2932.0,10.974523,12.05525,-inf,18.963005,172.0,0.0,2.94682,inf
3,TG_A_J_20190104,2019-01-04,4,0,0,25000.0,1408.0,TG_A_J,2019,1,...,1513.516129,2932.0,10.974523,12.05525,-inf,18.963005,172.0,0.0,2.94682,inf
4,TG_A_J_20190105,2019-01-05,4,0,0,32352.0,1250.0,TG_A_J,2019,1,...,1513.516129,2932.0,10.974523,12.05525,-inf,18.963005,172.0,0.0,2.94682,inf


In [57]:
train_df['week'] = train_df['week'].astype(float)
train_df['cos_week_of_year'] = train_df['cos_week_of_year'].astype(float)
train_df['sin_week_of_year'] = train_df['sin_week_of_year'].astype(float)

test_df['week'] = test_df['week'].astype(float)
test_df['cos_week_of_year'] = test_df['cos_week_of_year'].astype(float)
test_df['sin_week_of_year'] = test_df['sin_week_of_year'].astype(float)

In [58]:
# 10분위수로 카테고리화 하기

# 10분위수로 나누기

train_df['price_category'] = pd.qcut(train_df['price'], q=10, labels=False, duplicates='drop')
train_df['supply_category'] = pd.qcut(train_df['supply'], q=10, labels=False, duplicates='drop')


# 각 구간의 평균값 계산
mean_values = train_df.groupby('price_category')['price'].mean()
train_df['price_category_mean'] = train_df['price_category'].map(mean_values)

# 각 구간의 평균값 계산
mean_values = train_df.groupby('supply_category')['supply'].mean()
train_df['supply_category_mean'] = train_df['supply_category'].map(mean_values)


In [59]:
# 훈련 데이터에서 각 조합에서의 최빈 구간을 찾아서 매핑
item_price_mode = train_df.groupby(['item', 'month_ori','day_of_week'])['price_category'].apply(lambda x: x.mode().iloc[0])
item_supply_mode = train_df.groupby(['item','month_ori','day_of_week'])['supply_category'].apply(lambda x: x.mode().iloc[0])

test_df['price_category'] = test_df.apply(lambda row: item_price_mode.get((row['item'], row['month_ori'], row['day_of_week']), None), axis=1)
test_df['supply_category'] = test_df.apply(lambda row: item_supply_mode.get((row['item'], row['month_ori'], row['day_of_week']), None), axis=1)

mean_values_price = train_df.groupby('price_category')['price'].mean()
mean_values_supply = train_df.groupby('supply_category')['supply'].mean()

# 테스트 데이터에 평균값 추가
test_df['price_category_mean'] = test_df['price_category'].map(mean_values_price)
test_df['supply_category_mean'] = test_df['supply_category'].map(mean_values_supply)


In [60]:
train_df['price_category_mean'] = np.log(train_df['price_category_mean'])
train_df['supply_category_mean'] = np.log(train_df['supply_category_mean'])

test_df['price_category_mean'] = np.log(test_df['price_category_mean'])
test_df['supply_category_mean'] = np.log(test_df['supply_category_mean'])

In [72]:
international

,item,exWeight,exValue,imWeight,imValue,tradeBalance,category,year,month_ori
0,토마토(신선한 것이나 냉장한 것으로 한정한다),356571,990,0,0,990,0,2019,1
1,양파,821330,222,4003206,1118,-896,0,2019,1
2,쪽파,60,1,93405,128,-127,0,2019,1
3,꽃양배추와 브로콜리(broccoli),160,1,638913,563,-562,0,2019,1
4,방울다다기 양배추,0,0,7580,38,-38,0,2019,1
...,...,...,...,...,...,...,...,...,...
1269,포포(papaw)[파파야(papaya)],0,0,23830,71,-71,0,2023,2
1270,사과,135165,351,0,0,351,0,2023,2
1271,배,2206012,5411,1,0,5411,0,2023,2
1272,신 체리[프루너스 체라서스(Prunus cerasus)],5,0,0,0,0,0,2023,2


# AutoGluon

In [61]:
train_dropcols = ['ID', 'supply', 'month_of_year']
test_dropcols = ['ID', 'month_of_year']

In [62]:
# train_df_tg = train_df_tg.drop(columns = train_dropcols)
# test_df_tg = test_df_tg.drop(columns = test_dropcols)

# train_df_bc = train_df_bc.drop(columns = train_dropcols)
# test_df_bc = test_df_bc.drop(columns = test_dropcols)

# train_df_rd = train_df_rd.drop(columns = train_dropcols)
# test_df_rd = test_df_rd.drop(columns = test_dropcols)

# train_df_cr = train_df_cr.drop(columns = train_dropcols)
# test_df_cr = test_df_cr.drop(columns = test_dropcols)

# train_df_cb = train_df_cb.drop(columns = train_dropcols)
# test_df_cb = test_df_cb.drop(columns = test_dropcols)

In [63]:
train_ft = train_df.drop(columns = train_dropcols)
test_ft = test_df.drop(columns = test_dropcols)

In [64]:
train_ft

,timestamp,item,corporation,location,price,item_id,year,month_ori,week,day,...,imWeight,imValue,tradeBalance,category,exPrice,imPrice,price_category,supply_category,price_category_mean,supply_category_mean
0,2019-01-01,4,0,0,0.0,TG_A_J,2019,1,1.0,1,...,-inf,18.963005,172.0,0.0,2.94682,inf,0,0,0.153236,-1.686708
1,2019-01-02,4,0,0,0.0,TG_A_J,2019,1,1.0,2,...,-inf,18.963005,172.0,0.0,2.94682,inf,0,0,0.153236,-1.686708
2,2019-01-03,4,0,0,1728.0,TG_A_J,2019,1,1.0,3,...,-inf,18.963005,172.0,0.0,2.94682,inf,2,4,7.331624,11.520248
3,2019-01-04,4,0,0,1408.0,TG_A_J,2019,1,1.0,4,...,-inf,18.963005,172.0,0.0,2.94682,inf,2,4,7.331624,11.520248
4,2019-01-05,4,0,0,1250.0,TG_A_J,2019,1,1.0,5,...,-inf,18.963005,172.0,0.0,2.94682,inf,2,4,7.331624,11.520248
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,2023-02-27,3,5,0,468.0,RD_F_J,2023,2,9.0,27,...,0.0,0.000000,0.0,0.0,0.00000,0.0,1,4,6.343372,11.520248
59393,2023-02-28,3,5,0,531.0,RD_F_J,2023,2,9.0,28,...,0.0,0.000000,0.0,0.0,0.00000,0.0,1,4,6.343372,11.520248
59394,2023-03-01,3,5,0,574.0,RD_F_J,2023,3,9.0,1,...,0.0,0.000000,0.0,0.0,0.00000,0.0,1,4,6.343372,11.520248
59395,2023-03-02,3,5,0,523.0,RD_F_J,2023,3,9.0,2,...,0.0,0.000000,0.0,0.0,0.00000,0.0,1,4,6.343372,11.520248


In [65]:
train_ft.columns

Index(['timestamp', 'item', 'corporation', 'location', 'price', 'item_id',
       'year', 'month_ori', 'week', 'day', 'day_of_week', 'holiday',
       'cos_year', 'sin_year', 'cos_week_of_year', 'sin_week_of_year',
       'month_price_mean', 'month_price_max', 'exWeight', 'exValue',
       'imWeight', 'imValue', 'tradeBalance', 'category', 'exPrice', 'imPrice',
       'price_category', 'supply_category', 'price_category_mean',
       'supply_category_mean'],
      dtype='object')

In [66]:
# from autogluon.tabular import TabularPredictor
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
data = TimeSeriesDataFrame(train_ft)

In [67]:
SEED

42

In [69]:
predictor = TimeSeriesPredictor(
    prediction_length=28,
    target="price",
    eval_metric="RMSE"
)

time_limit = 30
# seed 고정
predictor.fit(data, time_limit=time_limit, random_seed=SEED)


No path specified. Models will be saved in: "AutogluonModels/ag-20231114_015708/"
================ TimeSeriesPredictor ================
TimeSeriesPredictor.fit() called
Setting presets to: ['best_quality']
Fitting with arguments:
{'enable_ensemble': True,
 'evaluation_metric': 'RMSE',
 'excluded_model_types': None,
 'hyperparameter_tune_kwargs': {'num_trials': 3,
                                'scheduler': 'local',
                                'searcher': 'auto'},
 'hyperparameters': 'best_quality',
 'num_val_windows': 1,
 'prediction_length': 28,
 'random_seed': 42,
 'target': 'price',
 'time_limit': 3600,
 'verbosity': 2}
Provided training data set with 59397 rows, 39 items (item = single time series). Average time series length is 1523.0. Data frequency is 'D'.
AutoGluon will save models to AutogluonModels/ag-20231114_015708/
AutoGluon will gauge predictive performance using evaluation metric: 'RMSE'
	This metric's sign has been flipped to adhere to being 'higher is better'. The

  0%|          | 0/3 [00:00<?, ?it/s]

	Trained 1 models while tuning DeepAR.
	-997.4773     = Validation score (-RMSE)
	413.52  s     = Total tuning time
Hyperparameter tuning model: TemporalFusionTransformer. Tuning model for up to 359.99s of the 3599.90s remaining.
	nan           = Validation score (-RMSE)
	377.06  s     = Training runtime
	4.48    s     = Validation (prediction) runtime
Hyperparameter tuning model: PatchTST. Tuning model for up to 359.99s of the 3599.90s remaining.
	-1105.7772    = Validation score (-RMSE)
	361.56  s     = Training runtime
	0.51    s     = Validation (prediction) runtime
Hyperparameter tuning model: DirectTabular. Tuning model for up to 359.99s of the 3599.90s remaining.
	-1076.5231    = Validation score (-RMSE)
	51.83   s     = Training runtime
	7.28    s     = Validation (prediction) runtime
Hyperparameter tuning model: AutoARIMA. Tuning model for up to 359.99s of the 3599.90s remaining.
	-845.6888     = Validation score (-RMSE)
	0.09    s     = Training runtime
	270.19  s     = Valid

# 예측

In [71]:
leaderboard = predictor.leaderboard(data)
leaderboard

Additional data provided, testing on additional data. Resulting leaderboard will be sorted according to test score (`score_test`).


                        model   score_test    score_val  pred_time_test  pred_time_val  fit_time_marginal  fit_order
0            WeightedEnsemble  -745.841228  -745.841228       89.694470      99.011815           5.109116         11
1                AutoETS_FULL  -797.059173          NaN       61.183923            NaN           0.000296         15
2                     AutoETS  -797.059173  -797.059173       29.556115      30.718363           0.086731          4
3                  Theta_FULL  -838.834688          NaN       97.235618            NaN           0.000343         14
4                       Theta  -838.834688  -838.834688       31.968705      48.663544           0.087925          3
5          SeasonalNaive_FULL  -843.925825          NaN        0.259777            NaN           0.000320         13
6               SeasonalNaive  -843.925825  -843.925825        0.278891       0.169497           0.096085          2
7                   AutoARIMA  -845.688839  -845.688839      253

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time_marginal,fit_order
0,WeightedEnsemble,-745.841228,-745.841228,89.694470,99.011815,5.109116,11
1,AutoETS_FULL,-797.059173,NaN,61.183923,NaN,0.000296,15
2,AutoETS,-797.059173,-797.059173,29.556115,30.718363,0.086731,4
3,Theta_FULL,-838.834688,NaN,97.235618,NaN,0.000343,14
4,Theta,-838.834688,-838.834688,31.968705,48.663544,0.087925,3
5,SeasonalNaive_FULL,-843.925825,NaN,0.259777,NaN,0.000320,13
6,SeasonalNaive,-843.925825,-843.925825,0.278891,0.169497,0.096085,2
7,AutoARIMA,-845.688839,-845.688839,253.912875,270.192143,0.087580,10
8,RecursiveTabular,-857.766628,-857.766628,1.818443,2.775150,367.192369,5
9,Naive_FULL,-997.063356,NaN,0.261182,NaN,0.000289,12


In [70]:
# 모델 재학습
predictor.refit_full()

Refitting models via `refit_full` using all of the data (combined train and validation)...
	Models trained in this way will have the suffix '_FULL' and have NaN validation score.
	This process is not bound by time_limit, but should take less time than the original `fit` call.
Fitting model: Naive_FULL | Skipping fit via cloning parent ...
Fitting model: SeasonalNaive_FULL | Skipping fit via cloning parent ...
Fitting model: Theta_FULL | Skipping fit via cloning parent ...
Fitting model: AutoETS_FULL | Skipping fit via cloning parent ...
Fitting model: RecursiveTabular_FULL



KeyboardInterrupt



In [ ]:
pred = predictor.predict(data, random_seed=SEED)
pred

In [ ]:
pred = pred.reset_index()['mean']
pred

In [ ]:

# 마이너스 값 0으로 치환
pred[pred < 0] = 0

# 제출

In [ ]:
submission

In [ ]:
submission['answer'] = pred
submission

In [ ]:
submission.to_csv(f"{DATA_PATH}submission_v5_1114.csv", index=False)
submission.to_csv("submission_v5_1114.csv", index=False)